In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
import pandas as pd

In [3]:
import json
data = []
with open('/content/train2.json') as f:
    for line in f:

            data.append(json.loads(line))


df = pd.DataFrame(data)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  2334 non-null   int64 
 1   headline      2334 non-null   object
 2   article_link  2334 non-null   object
dtypes: int64(1), object(2)
memory usage: 54.8+ KB


In [5]:
df['is_sarcastic'].value_counts()

,count
is_sarcastic,
0,1202
1,1132


In [6]:
samples = df[['headline']].values

In [7]:
samples

array([['thirtysomething scientists unveil doomsday clock of hair loss'],
       ['dem rep. totally nails why congress is falling short on gender, racial equality'],
       ['eat your veggies: 9 deliciously different recipes'],
       ...,
       ["skin in the game: why republicans' ahca bill should fail"],
       ['south sudan marks 6 years of independence as 6 million go hungry'],
       ['betsy devos sued for weakening sexual assault reporting protections for students']],
      dtype=object)

In [8]:
labels = df[['is_sarcastic']].values

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(samples, labels, test_size=0.2, random_state=42)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000,
                               output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(samples).batch(128)
vectorizer.adapt(text_ds)

In [12]:
vectorizer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('to'),
 np.str_('of'),
 np.str_('the'),
 np.str_('in'),
 np.str_('for'),
 np.str_('a'),
 np.str_('on'),
 np.str_('and')]

In [13]:
output = vectorizer([["thirtysomething scientists unveil doomsday clock of hair loss"]])
output.numpy()[0]

array([3263,  240, 1153, 6174, 6645,    3,  487, 1367,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [14]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [15]:
test = ["thirtysomething", "scientists", "unveil", "doomsday", "clock", "of", "hair", "loss"]
[word_index[w] for w in test]

[3263, 240, 1153, 6174, 6645, 3, 487, 1367]

In [16]:



!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-07-30 17:09:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-07-30 17:09:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-07-30 17:09:38--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [17]:
import os
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "/content/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [18]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 7069 words (449 misses)


In [19]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [20]:
class_names = set(labels.flat) # output shape i..e classes
class_names

{np.int64(0), np.int64(1)}

##LSTM

In [ ]:
from tensorflow.keras import layers
# Input Layer creation
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
# First Hidden Layer
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(1)(embedded_sequences) # 10 , 50, 200, 500
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 1)              │           408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,612 (7.63 MB)

 Trainable params: 412 (1.61 KB)

 Non-trainable params: 2,000,200 (7.63 MB)

In [ ]:
x_train = vectorizer(np.array([s for s in samples])).numpy()
x_val = vectorizer(np.array([s for s in samples])).numpy()

y_train = np.array(labels)
y_val = np.array(labels)

In [ ]:
x_train[:2]

array([[17011,   339,  3114,  7072,  2427,     3,   677,  1152,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
224/224 ━━━━━━━━━━━━━━━━━━━━ 21s 84ms/step - acc: 0.5215 - loss: 0.6926 - val_acc: 0.5236 - val_loss: 0.6920
Epoch 2/2
224/224 ━━━━━━━━━━━━━━━━━━━━ 18s 82ms/step - acc: 0.5242 - loss: 0.6920 - val_acc: 0.5236 - val_loss: 0.6920


In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(1, return_sequences=True )(embedded_sequences) # 10 , 50, 200, 500
x = layers.LSTM(1)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     2,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, None, 1)        │           408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 1)              │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,624 (7.63 MB)

 Trainable params: 424 (1.66 KB)

 Non-trainable params: 2,000,200 (7.63 MB)

In [ ]:
x_train = vectorizer(np.array([s for s in samples])).numpy()
x_val = vectorizer(np.array([s for s in samples])).numpy()

y_train = np.array(labels)
y_val = np.array(labels)

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_val, y_val))


Epoch 1/2
224/224 ━━━━━━━━━━━━━━━━━━━━ 41s 166ms/step - acc: 0.5177 - loss: 0.6926 - val_acc: 0.5236 - val_loss: 0.6920
Epoch 2/2
224/224 ━━━━━━━━━━━━━━━━━━━━ 31s 139ms/step - acc: 0.5260 - loss: 0.6918 - val_acc: 0.5236 - val_loss: 0.6920


##BERT


In [21]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [22]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
def tokenize_function(examples):
    return tokenizer(examples['headline'], padding='max_length', truncation=True)

In [24]:
tokenized_data = tokenizer(df['headline'].tolist(), padding='max_length', truncation=True)

data_list = []
for i in range(len(df)):
    data_list.append({
        'input_ids': tokenized_data['input_ids'][i],
        'attention_mask': tokenized_data['attention_mask'][i],
        'labels': df['is_sarcastic'].iloc[i]
    })

dataset = Dataset.from_list(data_list)

In [25]:
from transformers import TrainingArguments

In [26]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none" # Disable wandb logging
)

In [27]:
from transformers import Trainer

In [ ]:
from sklearn.model_selection import train_test_split

# Convert Dataset back to pandas DataFrame for splitting
dataset_df = dataset.to_pandas()

# Split the DataFrame using sklearn
train_df, eval_df = train_test_split(dataset_df, test_size=0.2, random_state=42)

# Create new Dataset objects from the split DataFrames
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.317500


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
trainer.evaluate()